## sample code for POX controller ##

In [ ]:
##RUN CODE ./pox.py log.level --debug your_pox_controller.py
##

from pox.core import core
from pox.lib.packet import ethernet, ipv4, tcp
from pox.lib.addresses import EthAddr, IPAddr
from pox.openflow import *
import time
import csv

log = core.getLogger()

class SimpleSwitch (object):
    def __init__ (self, connection):
        self.connection = connection
        connection.addListeners(self)

    def _send_data_packet(self, datapath, packet_data):
        # Build a TCP packet with dataset as payload
        pkt = ethernet.ethernet(dst=EthAddr("ff:ff:ff:ff:ff:ff"), src=EthAddr("00:00:00:00:00:01"))
        pkt.type = ethernet.ETH_TYPE_IP
        ip_packet = ipv4.ipv4(srcip=IPAddr("10.0.0.1"), dstip=IPAddr("10.0.0.2"))
        ip_packet.protocol = ipv4.IP_PROTOCOL_TCP
        tcp_packet = tcp.tcp()
        tcp_packet.srcport = 12345  # Source port
        tcp_packet.dstport = 80  # Destination port
        tcp_packet.payload = packet_data
        
        ip_packet.payload = tcp_packet
        pkt.payload = ip_packet
        datapath.send_packet_out(packet=pkt)

    def _handle_ConnectionUp(self, event):
        self.connection = event.connection
        log.info("Connection established: %s", self.connection)

        # Simulate transmitting dataset rows as TCP payload
        dataset = [
            "Row 1: data1, data2, data3",
            "Row 2: data4, data5, data6",
            "Row 3: data7, data8, data9"
        ]
        ]

        for row in dataset:
            self._send_data_packet(self.connection, row.encode('utf-8'))
            time.sleep(1)  # Wait a bit before sending the next row

    def _handle_PacketIn(self, event):
        packet = event.parsed
        if isinstance(packet.payload, ipv4.ipv4) and isinstance(packet.payload.payload, tcp.tcp):
            tcp_payload = packet.payload.payload.payload
            self.capture_packet_features(packet)
            
    def capture_packet_features(self, packet):
        # Capture packet features and write to CSV
        src_ip = packet.payload.srcip
        dst_ip = packet.payload.dstip
        src_port = packet.payload.payload.srcport
        dst_port = packet.payload.payload.dstport
        payload = str(packet.payload.payload.payload)

        # Write packet features to CSV file
        with open('captured_data.csv', 'a', newline='') as csvfile:
            fieldnames = ['Source IP', 'Destination IP', 'Source Port', 'Destination Port', 'Payload']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            # Check if the file is empty (i.e., it's the first time writing)
            if csvfile.tell() == 0:
                writer.writeheader()
            writer.writerow({
                'Source IP': src_ip,
                'Destination IP': dst_ip,
                'Source Port': src_port,
                'Destination Port': dst_port,
                'Payload': payload
            })
            log.info(f"Captured features: {src_ip} -> {dst_ip} | Payload: {payload}")

def launch():
    core.openflow.addListenerByName("ConnectionUp", SimpleSwitch)
    core.openflow.addListenerByName("PacketIn", SimpleSwitch)
    log.info("POX Controller running...")
